## Pandas 怎样新增数据列?
在进行数据分析时, 经常需要按照一定条件创建新的数据列, 然后进行进一步分析.

1. 直接赋值
2. df.apply方法
3. df.assign方法
4. 按条件选择分组分别赋值

In [1]:
import pandas as pd

## 0. 读取CSV数据到dataframe

In [2]:
fpath = "./datas/beijing_tianqi/beijing_tianqi_2018.csv"
df = pd.read_csv(fpath)
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3℃,-6℃,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2℃,-5℃,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2℃,-5℃,多云,北风,1-2级,28,优,1
3,2018-01-04,0℃,-8℃,阴,东北风,1-2级,28,优,1
4,2018-01-05,3℃,-6℃,多云~晴,西北风,1-2级,50,优,1


## 1. 直接赋值的方法
实例: 清理温度列, 变成数字类型

In [3]:
# 替换掉温度的后缀
df.loc[:,'bWendu'] = df['bWendu'].str.replace('℃', '').astype('int32')
df.loc[:,'yWendu'] = df['yWendu'].str.replace('℃', '').astype('int32')

In [4]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1


In [5]:
# 实例: 计算温差
# 注意, df['bWendu'] 其实是一个Series, 后面的减法返回的是Series 
df.loc[:, 'wencha'] = df['bWendu'] - df['yWendu']

In [6]:
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9


## 2. df.apply方法
实例: 添加一列温度类型
1. 如果最高温度大于33度就是高温
2. 低于-10度是低温
3. 否则是常温

In [7]:
def get_wendu_type(x):
    if x['bWendu'] > 30:
        return '高温'
    if x['yWendu'] < -10:
        return '低温'
    return '常温'

# 注意需要设置axis = 1, 这是Series的index是columns
df.loc[:,'wendu_type'] = df.apply(get_wendu_type, axis=1)
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha,wendu_type
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9,常温
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7,常温
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7,常温
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8,常温
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9,常温


In [8]:
df['wendu_type'].value_counts()

常温    286
高温     71
低温      8
Name: wendu_type, dtype: int64

## 3. df.assign方法
实例: 将温度从摄氏度变成华氏度

In [12]:
df.assign(
    yWendu_huashi = lambda x : x['yWendu'] * 9 /  5 + 32,
    bWendu_huashi = lambda x : x['bWendu'] * 9 / 5 + 32
)

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha,wendu_type,yWendu_huashi,bWendu_huashi
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9,常温,21.2,37.4
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7,常温,23.0,35.6
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7,常温,23.0,35.6
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8,常温,17.6,32.0
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9,常温,21.2,37.4
5,2018-01-06,2,-5,多云~阴,西南风,1-2级,32,优,1,7,常温,23.0,35.6
6,2018-01-07,2,-4,阴~多云,西南风,1-2级,59,良,2,6,常温,24.8,35.6
7,2018-01-08,2,-6,晴,西北风,4-5级,50,优,1,8,常温,21.2,35.6
8,2018-01-09,1,-8,晴,西北风,3-4级,34,优,1,9,常温,17.6,33.8
9,2018-01-10,-2,-10,晴,西北风,1-2级,26,优,1,8,常温,14.0,28.4


## 按条件选择分组分别赋值
按条件先选择数据, 然后对这部分数据赋值新列
实例: 高低温差大于10度, 则认为温差大

In [16]:
# 先创建空列
df['wenncha_type'] = ''

df.loc[df['bWendu']-df['yWendu']>10 , 'wenncha_type'] = '温差大'
df.loc[df['bWendu']-df['yWendu']<=10, 'wenncha_type'] =  '温差正常'
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,wencha,wendu_type,wenncha_type
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,9,常温,温差正常
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,7,常温,温差正常
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,7,常温,温差正常
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,8,常温,温差正常
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,9,常温,温差正常
